# VisitKorea Web Crawler

이 노트북은 `https://korean.visitkorea.or.kr` 사이트의 여행지 정보를 크롤링합니다.

## 주요 기능
1. 목록 페이지에서 상세 페이지 링크 추출
2. 상세 페이지에서 제목, 메타 정보(설명, 키워드), 이미지, 주소 추출
3. `time.sleep`을 이용한 랜덤 딜레이 적용 (1~5초)

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urljoin

## 상세 페이지 크롤링 함수

개별 여행지 상세 페이지에서 필요한 정보를 추출하는 함수입니다.
- `topTitle`
- `meta[name="description"]` content
- `meta[name="keywords"]` content
- `img src` (모든 이미지)
- 주소 (`<li>주소...<span>`)

In [2]:
def crawl_detail_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 1. topTitle 추출
        # id="topTitle" 또는 class="topTitle" 등을 시도합니다.
        top_title = "N/A"
        if soup.find(id="topTitle"):
            top_title = soup.find(id="topTitle").get_text(strip=True)
        elif soup.find(class_="topTitle"):
            top_title = soup.find(class_="topTitle").get_text(strip=True)
            
        # 2. meta name="description" content 추출
        meta_desc = "N/A"
        meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
        if meta_desc_tag and 'content' in meta_desc_tag.attrs:
            meta_desc = meta_desc_tag['content']
            
        # 3. meta name="keywords" content 추출
        meta_keywords = "N/A"
        meta_keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
        if meta_keywords_tag and 'content' in meta_keywords_tag.attrs:
            meta_keywords = meta_keywords_tag['content']
            
        # 4. img src="url" 모든 이미지 추출
        images = []
        for img in soup.find_all('img'):
            src = img.get('src')
            if src:
                # 상대 경로인 경우 절대 경로로 변환
                full_src = urljoin(url, src)
                images.append(full_src)
                
        # 5. 주소 추출 (<li> 중 "주소" 텍스트 포함하는 요소의 <span>)
        address = "N/A"
        for li in soup.find_all('li'):
            # 텍스트에 '주소'가 포함되어 있는지 확인
            if "주소" in li.get_text():
                span = li.find('span')
                if span:
                    address = span.get_text(strip=True)
                break
                
        return {
            "url": url,
            "title": top_title,
            "description": meta_desc,
            "keywords": meta_keywords,
            "address": address,
            "images_count": len(images),
            "images": images[:5] # 편의상 첫 5개만 출력 리스트에 포함 (전체는 필요 시 images 사용)
        }

    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None

## 목록 페이지 및 실행 로직

목록 페이지에서 상세 페이지 링크를 수집하고, 각 링크에 대해 크롤링을 수행합니다.

In [3]:

# 1. 목록 페이지 설정
list_url = "https://korean.visitkorea.or.kr/main/area_list.do?type=Place"
detail_base_pattern = "ms_detail.do"

# 크롤링할 대상 URL 리스트
target_urls = []

# 예시 URL 추가 (리스트 페이지 크롤링 실패 시 실행 확인용)
example_url = "https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=01b32883-1770-499e-9e0d-576736a55fbe"
target_urls.append(example_url)

print(f"Searching for links in: {list_url}")
try:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    # 목록 페이지 요청
    resp = requests.get(list_url, headers=headers)
    resp.raise_for_status()
    list_soup = BeautifulSoup(resp.text, 'html.parser')
    
    # 링크 추출 시도
    found_links = 0
    for a in list_soup.find_all('a', href=True):
        href = a['href']
        if detail_base_pattern in href:
            full_url = urljoin(list_url, href)
            if full_url not in target_urls:
                target_urls.append(full_url)
                found_links += 1
                
    print(f"Found {found_links} additional links from the list page.")
    if found_links == 0:
        print("Note: No links found automatically. The page might be loading data via JavaScript/AJAX.")
        print("Proceeding with the example URL provided.")

except Exception as e:
    print(f"Error getting list page: {e}")
    print("Proceeding with the example URL provided.")

print(f"Total URLs to crawl: {len(target_urls)}")

# 2. 크롤링 실행 Loop
crawled_data = []

for i, url in enumerate(target_urls):
    print(f"[{i+1}/{len(target_urls)}] Crawling: {url}")
    
    data = crawl_detail_page(url)
    
    if data:
        crawled_data.append(data)
        print(f"  -> Title: {data['title']}")
        print(f"  -> Address: {data['address']}")
        print(f"  -> Description len: {len(data['description'])}")
    
    # 3. Time Sleep (1~5초 랜덤)
    if i < len(target_urls) - 1:
        sleep_time = random.uniform(1, 5)
        print(f"  -> Sleeping for {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

print("\nAll crawling finished.")

Searching for links in: https://korean.visitkorea.or.kr/main/area_list.do?type=Place
Found 0 additional links from the list page.
Note: No links found automatically. The page might be loading data via JavaScript/AJAX.
Proceeding with the example URL provided.
Total URLs to crawl: 1
[1/1] Crawling: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=01b32883-1770-499e-9e0d-576736a55fbe
  -> Title: 
  -> Address: 우편번호 찾기
  -> Description len: 249

All crawling finished.


In [ ]:
✓ 나리농원 (ID: 01b32883-1770-499e-9e0d-576736a55fbe)
  ✓ 송정역(구) (ID: 024c67ec-878f-4b3a-9558-d4fc53d0d0b5)
  ✓ 위양지(위양지 이팝나무) (ID: 02bd0e4b-4f91-4d2c-9d20-2aa9c2027dfe)
  ✓ 이순신광장 (ID: 02d5ad3e-9d68-4812-842a-b3a1cbb2f7cb)
  ✓ 일산호수공원 (ID: 02e24733-9e25-4f84-9d19-09c68a7a75f5)
  ✓ 하동 화개장터 (ID: 0313c724-da5f-4095-9fb2-3b9aaf7fb8ce)
  ✓ 시몬스 테라스점 (ID: 03226c27-8605-475b-b7fe-025ad73f5369)
  ✓ 여수예술랜드 (ID: 0be2ef8f-0682-4564-8170-97b5bc8b4a65)
  ✓ 상주은모래비치 (ID: 0c70bb97-5ea0-4f1c-99e8-9a8cad34a596)
  ✓ 사려니숲길 (ID: 0fbb88c4-98a2-40e7-a679-23fc19107ce0)
  ✓ 고성 통일전망타워 (ID: 109ff263-0a52-4292-bcd3-e742d19f987e)
  ✓ 간월암(서산) (ID: 11c0ad01-6cb2-4855-abed-ff10fc20fd0e)
  ✓ 이순신공원 (ID: 1274035e-f6be-453a-91df-6c3b791043b5)
  ✓ 바람새마을 소풍정원 (ID: 12e2d0aa-f9e1-4f62-966e-bbcf359bc810)
  ✓ 죽녹원 (ID: 1424238b-7957-4660-8cd7-ef5742808ded)
  ✓ 상림공원 (ID: 204e4a3f-2e67-453d-9883-f60d4a0a5059)
  ✓ 베어트리파크 (ID: 20edc31a-5872-4da3-a387-740fde3d3fee)
  ✓ 서울랜드 (ID: 20f8834a-5ce5-45b2-b278-f038f71b8019)
  ✓ 해동용궁사 (ID: 2116b628-ded9-42c2-b17e-99d17b5b65e4)
  ✓ 대구 서문시장 & 서문시장 야시장 (ID: 214a6b35-763e-4071-bba7-63dbaa3dd72d)
  ✓ 서대문형무소역사관 (ID: 2160d763-5d20-4c8c-b234-1dae5a5776fa)
  ✓ 파크로쉬 리조트앤웰니스 (PARK ROCHE Resort&Wellness) (ID: 2234b19a-533b-4aab-be7d-b7eb75c1a8c4)
  ✓ 광한루원 (ID: 27ed68b5-c921-40d0-a8de-960d9a6b8939)
  ✓ 뮤지엄산 (ID: 2aa0e2bf-f20c-4961-936a-be7822ca882b)
  ✓ 해맞이해안로 (ID: 2b0316c0-ac57-4f86-9a07-6c12479e4ee3)
  ✓ 관방제림 (ID: 2cb8fa15-499c-466c-9094-73b5d5e818af)
  ✓ 표충사(밀양) (ID: 342f0ae0-4896-4174-b617-6a4a03e2f77f)
  ✓ 도담삼봉 (ID: 3531a546-e403-4a77-9feb-d28fd701601d)
  ✓ 비밀의 숲 (ID: 35f7511a-cb14-45af-8139-6754d8232fd2)
  ✓ 속초해수욕장 대관람차(속초아이) (ID: 36cf03de-005c-4bbe-9c23-ae29c0f92ebd)
  ✓ 해운대해수욕장 (ID: 36f54304-e9a8-4615-9226-eb749bf6ce75)
  ✓ 아쿠아플라넷 일산 (ID: 3a296d50-3ff8-43ec-94b5-ee3d58231aae)
  ✓ 대관령양떼목장 (ID: 3cc094b0-7a97-46cd-8649-87378cb24484)
  ✓ 옥연지 송해공원 (ID: 413b7a3a-82c1-47ac-98b2-c57b4d59bb7c)
  ✓ 안동 하회마을 [유네스코 세계유산] (ID: 3e375cc0-c11e-45de-9651-e77d6b40891e)
  ✓ 청풍호반케이블카 (ID: 4472cfa8-4d96-448f-acf6-05ef0147b8f6)
  ✓ 죽성드림세트장 (ID: 4ca61bd6-99ec-4828-bdc9-d295bec03a4d)
  ✓ 보은 법주사 [유네스코 세계유산] (ID: 4d6d9f37-1bf8-46a4-8ff6-82f408035acf)
  ✓ 충주 활옥동굴 (ID: 4f4129ba-d545-49c9-b099-9e2652d8a8ee)
  ✓ 제이드가든 (ID: 52920d1d-d953-4324-86cb-fc3dfd71c4e7)
  ✓ 테르메덴 (ID: 53466444-77b2-4ae7-9179-3867153c2ea5)
  ✓ 내소사(부안) (ID: 55203855-c891-4a23-83a4-1729ca70652b)
  ✓ 신세계백화점 본점 (ID: 5681a3b3-52a9-46a8-bbc7-a6a63f191cf1)
  ✓ 국립경주박물관 (ID: 56a55559-7cb2-4e5e-97b0-02de82ab79b2)
  ✓ 경포해수욕장 (ID: 60ad68a9-2222-4558-96d7-7f0b44736aef)
  ✓ 평택호관광단지 (ID: 6623c95c-7bcb-4658-befd-64986e895ba5)
  ✓ 울산 농수산물도매시장 (ID: 66479cb0-caeb-4d1f-a733-942b4ff57bd5)
  ✓ 전북 전주 한옥마을 [슬로시티] (ID: 66dba60b-a623-4753-a3b4-76a62fcbbe05)
  ✓ 단양장 (1, 6일) (ID: 67eef32f-3ce4-4860-b383-5ca0ba1859d2)
  ✓ 주문진수산시장 (ID: 683db658-614f-4924-9099-d758687660d6)
  ✓ 파라다이스 스파 도고 (ID: 6b82b7ae-ec7d-48da-8d24-8561dddaff26)
  ✓ 경주 대릉원 일원 (ID: 6da54421-9ab9-4853-80d6-1eadbf0ad73b)
  ✓ 순천만습지 (ID: 6e2df011-c197-41d5-b87a-38f96734fcf9)
  ✓ 태종대 (ID: 729fae7d-4312-40ad-9f5d-e470149e0026)
  ✓ 아르떼뮤지엄 강릉 (ID: 7484c081-80e2-4521-944b-bb09667641bb)
  ✓ 앞산빨래터공원 (ID: 76c45070-2f27-478f-9ece-303afe6e5147)
  ✓ 해인사(합천) (ID: 77872fac-c479-4f61-9475-9f172ed8a2c8)
  ✓ 롯데월드 어드벤처 (ID: 78a00ac3-ad3c-4637-8629-48aefb22a79b)
  ✓ 쏠비치 호텔&리조트 삼척 (ID: 78bbed1d-f549-4ed4-a438-b86ceeeb61cf)
  ✓ 통도사(양산) (ID: 7a1a6cd1-d3bf-4fd6-8ad7-f664f38d953c)
  ✓ 부소담악(추소정) (ID: 7b29ecdd-154b-41d7-a79c-7dbadf3ddd1b)
  ✓ 소노벨 단양 (ID: 7d657724-2938-4ad9-a01f-13e068311823)
  ✓ 아홉산숲 (ID: 833d4a7b-e9ef-4b2a-bfab-4e376620b458)
  ✓ 학동 흑진주 몽돌해변 (ID: 83b8305a-c8ec-48b3-aa7f-39da82ce7ced)
  ✓ 스누피가든 (ID: 84a049ea-cdd1-4413-9ce4-9acf1b24ed36)
  ✓ 밀락더마켓 (ID: 85d95ad6-84c2-44ff-bda1-3bffd1137ad6)
  ✓ 제주 서귀포 산방산 (ID: 86f54cfe-54b8-4160-9c05-8d1088392e2d)
  ✓ 여수 유월드 루지 테마파크 (ID: 8732969c-378e-4103-a0c2-dfd8db152992)
  ✓ 서동공원과 궁남지 (ID: 87a8fbad-70cb-4ce2-9a7c-a37578204fd3)
  ✓ 담양 메타프로방스 (ID: 87e12acf-cc0f-4fe8-b954-491169ef07bc)
  ✓ 송도 센트럴파크 (ID: 87e613c6-f7c3-4d3f-8d60-13c2f8166b95)
  ✓ 스타필드 안성 (ID: 88bc85ef-7185-4ebe-8c3b-02571a8cc1f2)
  ✓ 재인폭포 (한탄강 유네스코 세계지질공원) (ID: 88e97014-89a9-4cb2-82d6-71ca09319cbe)
  ✓ 삼성궁 (ID: 8fa5ea22-82a6-4041-a21e-af0b358820a6)
  ✓ 양평 두물머리 (ID: 925ba0a6-1d1b-42cf-a592-daaa01826e3d)
  ✓ 광안리해수욕장 (ID: 98763dd6-119d-4e32-910e-a40652334bda)
  ✓ 다대포해수욕장 (ID: 9ac39dd9-4dfe-4dc0-866e-b3a98fd581c7)
  ✓ 경주엑스포대공원 (ID: 9acbd5d3-91c6-40c3-b148-62b15dce9687)
  ✓ 더베이101 (ID: 9b235dcd-d347-4e00-9685-148e7c5a951e)
  ✓ 경주 첨성대 (ID: 9d506d09-62ec-40d4-87a2-a0fd35b25b80)
  ✓ 대왕암공원 (ID: 9e0f6d0d-ae33-4032-ac05-620137804c00)
  ✓ 팔공산 케이블카 (ID: 9f9e4d69-3f4b-423b-93e6-f675106d58ea)
  ✓ 만천하 스카이워크 (ID: 9ff81615-d49a-4fb8-8b4c-2cfd3ed239d6)
  ✓ 부평깡통시장 (ID: a0cbd123-4822-4e43-b8ef-e13bba8ff0f7)
  ✓ 옥정호 출렁다리 및 붕어섬생태공원 (ID: a2229f76-49c6-4e7a-b2e5-bfc96581a217)
  ✓ 단양 고수동굴 (ID: a3444f94-3c9f-4b97-8ea3-1aeff940dd39)
  ✓ 삼양라운드힐 (ID: a772dc50-e987-400b-b947-62e151b23e90)
  ✓ 부산 송도해수욕장 (ID: a8bd1acb-9547-4f3f-8695-9a94ba19b1f9)
  ✓ 청남대 (ID: a9e8babe-ff38-4065-82f8-0dfad023ec83)
  ✓ 호미곶 해맞이광장 (ID: ad0b6c37-e48d-4c64-9ed4-201c58d9ce57)
  ✓ 최참판댁 (ID: ae28c3e3-2121-4366-a271-8cb81be19792)
  ✓ 코엑스 (ID: ae7c4ce1-5629-4113-97f9-47cd1286e7ab)
  ✓ 부산 암남공원 (ID: aebb3990-a608-4400-b26e-58e744093245)
  ✓ 송정해수욕장 (ID: b0136743-e02d-4f1b-bef6-cc1b63536489)
  ✓ 경주 동궁과 월지 (ID: b046dbdc-67db-49f0-acca-fffa75befdf9)
  ✓ 일광해수욕장 (ID: b1baf4b5-acae-47be-a8c6-02c0aec129c7)
  ✓ 미리내성지 (ID: b78ad4da-f2af-43e8-bff9-b769b9826314)
  ✓ 아쿠아플라넷 여수 (ID: b7d41f83-443c-4c52-ab04-0e8d46edaf57)
  ✓ 강릉 경포대 (ID: babcea7b-a8e6-4a30-9e33-f783ad33fe79)
  ✓ 한국민속촌 (ID: bbb988ee-f8e9-43e7-8fdf-691cd78a68ba)
  ✓ 원대리 자작나무 숲 (속삭이는 자작나무 숲) (ID: bc23387a-1c9a-4a0a-86f6-ae779baae904)
  ✓ 아르떼뮤지엄 제주 (ID: be193377-7fd0-482b-a730-6a9c2cd6d4af)
  ✓ 용머리해안 (ID: c44ccaff-4c8e-4a73-a251-21b394b66cc2)
  ✓ 이월드 (ID: c560c170-53fc-425f-aea9-7443e764ea4b)
  ✓ 해운대시장 (ID: c818415f-0a57-486e-937b-79afa789ce45)
  ✓ 하늘공원 (ID: c8cdaa17-55fb-4340-b15a-4401cb8b4635)
  ✓ 영남알프스 얼음골케이블카 (ID: ca212214-4cd5-41a6-9d14-c96593ded08d)
  ✓ 마곡사 [유네스코 세계유산] (ID: ca77bec7-25e5-4a0b-9a37-1ca774319251)
  ✓ 강화 전등사 (ID: cb007a7c-ee6e-4f23-adfe-34f3adf1d19c)
  ✓ 부석사 [유네스코 세계유산] (ID: cb420683-215e-4a5c-89ec-b626e5df6612)
  ✓ 자라섬 (ID: d6acd121-1077-4b20-94f7-512cc975b01c)
  ✓ 단양 다누리아쿠아리움 (ID: d6f9ae85-97df-49fc-b0a8-d68e91ce809b)
  ✓ 남해독일마을 (ID: d79e9827-abf3-44b3-bc01-afd1ddd04017)
  ✓ 채석강 (전북 서해안 국가지질공원) (ID: d984f94a-b0d3-4e8e-b80e-843d8db64bf5)
  ✓ 죽변항 (ID: e0101ce9-6f94-4fd8-8c06-8cf18852dd31)
  ✓ 비자림 (ID: e0abf9cf-9865-4bf4-9077-93f2bb083a45)
  ✓ 추암 촛대바위 (ID: e13ee150-d204-49d3-890d-68f66dcfecce)
  ✓ 정동진해변 (ID: e42599c9-bac7-4f83-8a89-fe4b7464d90c)
  ✓ 낙산사 (ID: e4d4eb0f-61c5-4381-a914-a23bac522aed)
  ✓ 방화수류정(동북각루) (ID: e8317f05-41c5-4198-8779-90d7d258c7c5)
  ✓ 아침고요수목원 (ID: e8660b54-9a93-4e87-9bb8-6ff0cb934b19)
  ✓ 섬진강기차마을 (ID: e9200c46-98cf-423b-8706-55a1fb6dd7ac)
  ✓ 허브아일랜드 (ID: eab3ecb0-e57d-4a2f-8bf3-b5c03aebadf3)
  ✓ 장태산자연휴양림 (ID: ee78a56f-8fb0-421e-b714-b20472df9e00)
  ✓ 대전오월드 (ID: f033d915-a5ac-4c63-8680-481f1d06ba77)
  ✓ 화성행궁 (ID: f1767d38-fcd4-48e5-ac7b-977866fba906)
  ✓ 산막이옛길 (ID: f28a4a6d-481e-4895-bb4c-b03e3ecc7664)
  ✓ 하나개해수욕장 (ID: f29d8588-8775-4a27-868f-113241fbcb93)
  ✓ 청령포 (강원고생대 국가지질공원) (ID: f3ba9aa5-5f99-469d-8c2d-7fea776145d1)
  ✓ 천지연폭포 (ID: f4b66a09-b485-43c9-8162-c22993ad13c8)
  ✓ 설악산 케이블카 (ID: f5433080-87a1-46cf-bc2f-617f62ccba2c)
  ✓ 성산일출봉 [유네스코 세계자연유산] (ID: f5da2334-1c8c-4049-8e06-4db38333a57f)
  ✓ 영금정 (ID: faed64ba-2fcc-4409-a9f7-a1d64d4ceefa)
  ✓ 소노캄 경주 (ID: fe6e3425-7fae-4cc6-beeb-de0e4dd61531)
  ✓ 화엄사 (ID: f7e763b3-7616-4bb2-b85d-5c630977c374)
  ✓ 포천 산정호수 (ID: fed74f79-c3f1-4e3b-86f3-244c82c25306)
  ✓ 롯데프리미엄아울렛 의왕점 (ID: 17f83262-8f37-4769-926b-ae9a52874973)
  ✓ 망원한강공원 (ID: 03be5466-bda7-4dfb-8208-d4792f13cb07)
  ✓ 이촌한강공원 (ID: 17fa606c-5d84-48bb-b0e7-3826ce843c0b)
  ✓ 라발스호텔 (ID: 1b9670a4-96d9-4321-9a7d-05344b2a4b0d)
  ✓ 소노캄 거제 (ID: 21b2d0d8-f138-4f55-a8ac-5c1dd68a56e9)
  ✓ 신포국제시장 (ID: 28e67b66-ad86-458a-b1d1-42c120c07d29)
  ✓ 소노캄 고양 (ID: 39839196-a9f5-40a4-a921-76ad3e669a31)
  ✓ 안성팜랜드 (ID: 3bfbe9d0-fc7d-457b-8abc-ac75dc7ebdbd)
  ✓ 플라이웨이 케이블카 (ID: 429f4129-d6e7-421c-9fb9-96f36bbec9e3)
  ✓ 덕평공룡수목원 (ID: 52d663a5-064d-4222-adbc-b3104254384c)
  ✓ 대전역 동광장 (ID: 59d028ae-5a07-4ee1-845c-a514a6252549)
  ✓ 행주산성 역사공원 (ID: 6b57ccdd-4fb8-43ec-b89b-b607a41546e2)
  ✓ 독립기념관 (ID: 6c142ad1-eb98-4d60-ac17-986fae54f608)
  ✓ 금호리조트 제주 (ID: 78dc8e8c-d851-4eb0-8eb5-fb2a4b0cce7f)
  ✓ 월광수변공원 (ID: 8321115a-b175-47c0-8b6e-5ec7df11b70d)
  ✓ 웨스틴 조선 부산 (ID: 84e668e0-47d5-454a-bca7-f1e1daf81cd9)
  ✓ 백운포체육공원 (ID: 85902716-13d3-4fdc-acf3-850462d19107)
  ✓ 무주덕유산리조트 (ID: 9227511b-6c82-478c-9e68-9af077902720)
  ✓ 목포 연안여객선터미널 (ID: 93ee7428-473d-408a-b4a3-48654857dc28)
  ✓ 소노캄 여수 (ID: 986a56f3-6281-4e5f-89ff-b15519c679c6)
  ✓ 벽초지수목원 (ID: 9ffcf9ff-1b25-4abd-8268-6f3a0dc4c296)
  ✓ 롯데호텔서울 (ID: b128b1a1-ce82-40a7-84ef-6c00e390c10c)
  ✓ 대구삼성라이온즈파크 (ID: b1d56c6f-e89a-421f-a598-a2cfe727ff74)
  ✓ 캐리비안베이 (ID: b8bfbe35-883e-4b10-a07f-5ae36a30d359)
  ✓ 세종호수공원 (ID: d25bea6d-fce9-4dea-b98c-76b70932ec01)
  ✓ 쌍계사(하동) (ID: d6b47398-0d31-4f70-89fe-0fea5badf4d7)
  ✓ 제주신라호텔 (ID: e17edf26-d654-49da-bead-56729c32702a)
  ✓ 봉은사(서울) (ID: eba1cc70-dbed-4b31-b61b-5292d7eabe77)
  ✓ 서울 헌릉(태종·원경왕후)과 인릉(순조·순원황후) [유네스코 세계유산] (ID: dc1158ac-23a0-4102-90a4-0e43a4e333bc)     
  ✓ 그랜드 조선 부산 (ID: ee79549c-d670-41e5-8179-eb8f40c2781e)
  ✓ 신세계백화점 센텀시티점 (ID: f5e84adc-34e5-486d-8b34-318d2539d628)
  ✓ 현충사 (ID: f83608de-229c-43e3-ac7e-8b95fb3be0c0)
  ✓ 보라매공원 (ID: fbcaaedf-49e6-479d-99b0-e8b4c376e1c8)
  ✓ 송광사(순천) (ID: 0473c8f7-078c-4565-ae89-04bf0778209c)
  ✓ 잣향기푸른숲 (ID: 054833a5-5989-4b6e-81ef-e983d3ad93cd)
  ✓ 배론성지 (ID: 06532bda-250e-45ca-a802-9211a43f4c15)
  ✓ 해밀톤 호텔 (ID: 09de14d1-6177-44a2-8430-2fbf0fbe068e)
  ✓ 덕구온천 스파월드 (ID: 0e8c03ab-4ccd-4ba5-bd4d-3f43df027665)
  ✓ 안흥찐빵마을 (ID: 0f6d08be-6d22-4f76-a126-8d3da4ab4976)
  ✓ 동백포레스트 (ID: 10301f56-cc9d-4d90-88a4-5e9fbae4942d)
  ✓ 말티재 전망대(말티고개) (ID: 11c0b068-2d38-4833-8d21-ece92cc763f9)
  ✓ 등기산스카이워크 (ID: 12f740a3-2117-4f4e-a13f-cad34501279b)
  ✓ 가리왕산케이블카 (ID: 147b0c1b-f2b2-4323-8246-44465b2113d4)
  ✓ 제일유황온천 (ID: 1a528574-6991-402a-9688-6bdbf41ca90b)
  ✓ 자수정동굴나라썰매장 (ID: 2d2c5663-0ad7-4a33-82a1-1a19465f7944)
  ✓ 강릉 솔향수목원 (ID: 2f58a487-3f76-4d18-8fa8-d8a9536f2a19)
  ✓ 관문체육공원 (ID: 32c41d52-a9d3-4ce3-b56e-de02f8c3d963)
  ✓ 약천사(제주) (ID: 32cb406e-8135-43c2-adc6-7d0e92c87d91)
  ✓ 국립부여박물관 (ID: 34fb72fa-1227-4279-9794-8e4933ef96b9)
  ✓ 뿌리공원 (ID: 3732f9d0-1c50-45ef-81d6-0b25a83e86da)
  ✓ 평택국제중앙시장 (ID: 3940a6cb-dac0-437b-8b47-808dbdb1e05c)
  ✓ 곡교천 은행나무길 (ID: 39e1198c-0c52-46ae-8786-c7b2889c481f)
  ✓ 전주시 농수산물도매시장 (ID: 42cc1206-6d4f-49ea-b219-0f064f7a9d9a)
  ✓ 순천 드라마촬영장 (ID: 4e13f1db-7669-4d55-9e7c-7fdb61808a86)
  ✓ 대동백화점 (ID: 4f33fff1-7315-485e-9e29-a914d211b45d)
  ✓ 수타사 (ID: 578647da-774b-44a5-91cb-35b2d4a7b255)
  ✓ 연오랑세오녀테마공원 귀비고 (ID: 6647b8db-be82-41e7-b339-48142292ee33)
  ✓ 세운상가 (ID: 6c4c7c67-9e2b-487d-b67a-45f37f62bdbc)
  ✓ 오륙도 스카이워크 (ID: 6cb1642d-b578-462f-82c6-b1c1ef468af7)
  ✓ 고성공룡박물관 (ID: 6d1dd69f-5b8b-4f4e-9320-c2587720c65a)
  ✓ 담양 메타세쿼이아길 (ID: 6e6fad62-1c2a-43ec-8cd9-760e662d8fc7)
  ✓ 황지연못 (ID: 6f37e354-2b23-44c0-abab-34a6615e3aa9)
  ✓ 볼베어파크 롯데몰 은평점 (ID: 736d26b0-90b1-4542-8a62-1e8109e19388)
  ✓ 단양강 잔도 (ID: 78d14f23-dca9-43c3-82d9-cc715c1d8dd8)
  ✓ 롯데월드타워 서울스카이 (ID: 7e26f7f5-dde8-4eab-b2e9-42d6d37aded1)
  ✓ 북수원온천 (ID: 7ee69b57-3225-4f07-aee0-b8ffed549df3)
  ✓ 산청 동의보감촌 (ID: 8b2b5bff-3810-4d69-9a3a-0815b4271d64)
  ✓ 포항 여객선 터미널 (ID: 8b61d7be-d352-43b3-a91d-d22f7d4244c5)
  ✓ 모나 용평 스키장 (ID: 8f33ac94-4c12-40e4-acc8-707fd5f2b9c0)
  ✓ 은평한옥마을 (ID: 981f66d0-9d58-4b64-b8c1-96e28106f914)
  ✓ 청송 얼음골(청송 국가지질공원) (ID: 9bcb03e9-03fe-4ddf-b9d0-920dffc61fea)
  ✓ 벌영리 메타세콰이어길 (ID: 9d1a5779-c0be-4476-843c-151b805653ea)
  ✓ 대천해수욕장 스카이바이크 (ID: a55de666-28b0-48fd-80a6-de967a51cfe4)
  ✓ 라마다프라자 수원호텔 (ID: a63d9d38-665a-4e45-b9a6-2daa92530b87)
  ✓ 수선사 (ID: a6fa7b86-f5fe-4410-9749-75b2772665d5)
  ✓ 경기북부어린이박물관 (ID: a7306496-9bb5-4d37-80ae-1ec76b5c5398)
  ✓ 태권도원 (ID: aa0000ac-2fc5-4a0f-a626-8a2a0dab72ed)
  ✓ 횡성루지체험장 (ID: aa7e2677-0b32-4cbb-9684-cb6c6943a2a4)
  ✓ 수생식물학습원 (ID: b717375f-284e-44cc-a745-dac33ee4138b)
  ✓ 부산 감천문화마을 (ID: bfafd82a-6e0a-4455-bfef-10e478de5ca4)
  ✓ 진주성 (ID: c2e7414e-d4ab-46c0-aed2-39d4b3f55b56)
  ✓ 경주 버드파크 (ID: c9304c33-6373-4a6a-b0de-3ac3416760ee)
  ✓ 부산 어린이대공원 (ID: cad910d9-1ad4-4812-a4e0-22769290c226)
  ✓ 세계꽃식물원 (ID: cb8a5f4c-4192-4fc3-ac4a-87859b983448)
  ✓ 에델바이스 스위스 테마파크 (ID: ce6863ec-9384-4d33-aee9-81c34fb215ba)
  ✓ 청사포 다릿돌전망대 (ID: d24fc1d9-56ee-4cec-b8c5-8ba1f20101ee)
  ✓ 아르떼뮤지엄 여수 (ID: e278f02e-0194-49d9-8cfd-ad8dabc57380)
  ✓ 덕수궁 (ID: e413abf3-f8a7-486d-95fd-4fb12598aefc)
  ✓ 수주팔봉 (ID: e67ea2e0-7020-43b1-bc44-1f98537ec009)
  ✓ 무진정(함안) (ID: e8a3c954-7a1b-42b2-8bbc-f145f66d70b8)
  ✓ 월정교 (ID: ec28bc85-95ba-4b58-a36d-3f02988eeae1)
  ✓ 하피랜드 (ID: ecbe8f2c-2278-4024-a030-f0ede4a0f3bd)
  ✓ 국립중앙과학관 (ID: eea09e24-5043-489f-ab6a-7267f405f9e2)
  ✓ 제주동백수목원 (ID: f1f6a489-bd74-42a8-be35-afd75748aa00)
  ✓ 제주시민속오일시장 (2, 7일) (ID: f4493fc6-00fb-4dcf-a554-9d9899fb46c2)
  ✓ 민락어민활어직판장 (ID: fd3a7caa-fc19-4a9e-91fc-054e411599b4)
  ✓ 삼척해상케이블카 (ID: ffe647e0-5892-484a-a785-cc5ab971aea4)
  ✓ 서대문자연사박물관 (ID: 0047b203-d422-492e-bbe1-1554de519405)
  ✓ 신세계사이먼 부산 프리미엄 아울렛 (ID: 0097e8ba-126b-47b2-8280-51c55b1176c4)
  ✓ 수원시립미술관 (ID: 00c792fe-3855-4e92-a780-0f81c96f4165)
  ✓ 문경새재도립공원 (ID: 01245420-33b3-486d-8d6f-d3e5d4b4a20b)
  ✓ 설악해변 (ID: 01b1e622-5d2f-4789-be73-c5ca8cf6ac66)
  ✓ 대금굴 (대이리 동굴지대) (ID: 0213bf10-d3b9-4db0-bad9-1b13e3acded6)
  ✓ 서울숲 (ID: 02f56ab2-5e53-470f-8ec3-95cf8fb88aa4)
  ✓ 속초 외옹치 바다향기로 (ID: 034ff037-924e-4e15-8ad9-d0c0de59c2b8)
  ✓ 국립중앙박물관 (ID: 0363da87-ef3b-4c21-8508-34f70dda31fb)
  ✓ 용인자연휴양림 (ID: 04e30cb5-05f3-4d90-98fb-1b2bc4c95959)
  ✓ 경주 양남 주상절리 전망대 (ID: 051815f6-9d68-4901-8c2d-a9ce81a49fac)
  ✓ 상화원 (ID: 05da6212-b6ad-41f5-ab91-b4d66345184c)
  ✓ 울진 후포항 (ID: 06f751e6-0e4e-47f7-8798-fd7bbc8e2fb6)
  ✓ 한담해안산책로 (ID: 071a34b5-78f3-419b-8ef3-a987b535f443)
  ✓ 인천대공원 (ID: 0b2dec74-015b-4a3a-bd19-cb2a449720d4)
  ✓ 물미해안전망대 (ID: 0c8493e2-7172-49cf-ae82-fc4911f261ba)
  ✓ 통영케이블카 (ID: 0c8fdbdf-39d2-4e46-8403-e05d9c1bd168)
  ✓ 휴애리자연생활공원 (ID: 0f3410d7-e89f-45fe-acd2-05231552d59c)
  ✓ 알파카월드 (ID: 12a6670d-7251-49cd-b097-224cd308cdd1)
  ✓ 유달산일주도로 (ID: 12d6cf84-540e-4b5d-a2c3-bcbc9c8b0322)
  ✓ 속초해수욕장 (ID: 12e3b8dc-81ad-44e7-87af-ffd9adb412d6)
  ✓ 여수 해상케이블카 (ID: 1474d159-54e5-4e8c-862f-07d5c6021cdb)
  ✓ 경주월드 어뮤즈먼트 (ID: 158b5695-9c71-47ef-a2b3-939856efaae0)
  ✓ 안양예술공원 (ID: 1813ac6b-06e7-49c1-9092-b7c0ad1712a2)
  ✓ 파주 임진각(평화누리공원) (ID: 197e6e5e-b111-4561-9edc-2579205d9151)
  ✓ 연천 호로고루 (ID: 19c090e9-3a91-4bf7-9d3d-aa37eaaff273)
  ✓ 길상사(서울) (ID: 1a007b60-c892-46c3-94dc-c7051ac9fa4f)
  ✓ 지산 포레스트 리조트 스키장 (ID: 1abed7cc-ef27-4004-9b63-474a5d1dd6ec)
  ✓ 지중해마을 (ID: 1b54ecfa-81aa-4e5b-94dc-2c00a4fb0221)
  ✓ 국립현대미술관 청주 (ID: 1c95a4e1-22f7-4e91-8b8f-f157059eb83e)
  ✓ 강릉 중앙시장 (ID: 1e7926d8-8ed3-44ce-a3d2-05d437a631aa)
  ✓ 무창포해수욕장 (ID: 2211f2e7-72d4-4be6-afa2-188da2a22afa)
  ✓ 소양강스카이워크 (ID: 22addecf-a1ee-43c6-9436-3bc13baecb10)
  ✓ 가락몰 (ID: 236237b4-7528-4d25-955d-e985cb6f78fc)
  ✓ 광장시장 (ID: 242c556a-2365-47b9-be5c-ea36d8db3663)
  ✓ 북악스카이 팔각정 (ID: 2444850e-b6da-4a0e-a2dd-0ed4347e5361)
  ✓ 사천해변(사천해수욕장) (ID: 24b882a4-8eae-4bd3-890b-aaae26421c83)
  ✓ 한국도로공사 전주수목원 (ID: 2529b8db-ddeb-4bb1-a646-a71bdb5db637)
  ✓ 동학사(공주) (ID: 255b2ddd-964c-4366-ac67-935206df4d0a)
  ✓ 청주 육거리종합시장 (ID: 263be09c-2e8a-4124-be3d-be2abd207a36)
  ✓ 화개정원 (ID: 26785488-ed4b-4543-9669-90a240b7530c)
  ✓ 새별오름 (ID: 288ca75c-0b80-42a0-a775-bfd8c709c462)
  ✓ 교동도 대룡시장 (ID: 2a54e990-951a-4203-82eb-592be2ba3c99)
  ✓ 허심청 (ID: 2a56199d-48be-42da-8afb-8d0eaafb5d1e)
  ✓ 세화해변 (ID: 2bcd66e6-f1c5-44c7-b48f-28e861931124)
  ✓ 수성못 유원지 (ID: 2cd507a4-63fc-46dd-8abe-00d73539136a)
  ✓ 한화리조트 설악 워터피아 (ID: 2e2740db-34db-48ef-bb4a-5810d32b7442)
  ✓ 신소양체육공원 (ID: 2ef9aacc-2520-4bad-97c3-1ee9a5a7e198)
  ✓ 행주산성 (ID: 2f3031f7-c0b1-4688-9fb1-c2488d701616)
  ✓ 목포 해상케이블카 (ID: 303713a3-eafc-484c-b609-be44d4e76a3c)
  ✓ 천제연폭포 (ID: 32ccc98e-e00b-463b-80db-0e33cd20c86e)
  ✓ 광주 양동시장 (ID: 3378f93e-0d5f-433b-9026-8d2a780d155a)
  ✓ 태화강 국가정원 (ID: 34647178-fe80-4b2c-a0f4-0c372f7daf34)
  ✓ 금산 보리암(남해) (ID: 34d2e5c7-3c44-4e82-ab3b-831c3ae2b5be)
  ✓ 서울역사박물관 (ID: 36d0d7f2-3197-4bf3-a30e-562775f03e97)
  ✓ 영일대해수욕장 (ID: 37ac7bb5-e4b8-4cdf-8cbc-c20b1e98a558)
  ✓ 최참판댁 한옥숙박시설 (ID: 37fdef23-2238-405e-8388-badb31dfd686)
  ✓ 롯데월드 어드벤처 부산 (ID: 382a6366-a64c-4437-8a23-0fb5770a6858)
  ✓ 백양사 (ID: 3982fe50-8f81-46cb-b72b-c530a34884b4)
  ✓ 강촌레일파크(김유정레일바이크) (ID: 39930876-ab83-4e37-9a95-480205ccf9bc)
  ✓ 소래포구 (ID: 3aee6266-f4a5-4880-a79c-5181fa90c80a)
  ✓ 낙산공원 (ID: 3c98e24b-9fb9-4ecf-b8a2-f26a5890748f)
  ✓ 천안상록리조트 아쿠아피아 (ID: 3ea8fdc2-e1a3-491f-af49-2d11f53f92c6)
  ✓ 안면도자연휴양림 (ID: 3f083901-93ef-4b45-9610-f8a9867a37ab)
  ✓ 부산 자갈치시장 (ID: 3fdaaf29-b55d-4076-a06b-ba963a493174)
  ✓ 한담해변 (ID: 4058d457-c138-4cd6-abd0-b584495cb5f9)
  ✓ 국립생물자원관 (ID: 406f7510-f551-448c-a893-dfd1b683a461)
  ✓ 동피랑마을 (ID: 41211b90-1572-4fdf-9766-da12341150e6)
  ✓ 덕진공원 (ID: 4129a02b-5cdb-414f-8872-101082786428)
  ✓ 여수수산시장 (ID: 44b0e803-4f33-4635-8cf8-5c1bd3f0c695)
  ✓ 경복궁 (ID: 443caffd-74a6-4713-ba02-4b5520ef7e62)
  ✓ 망상해변 (ID: 46a1d5f9-c4f9-434b-879a-2d3f02330709)
  ✓ 애니포레 (ID: 48ed4b57-55ba-4ce8-84e2-f8593268985a)
  ✓ 충장로 (ID: 4c60a0ab-2e15-416a-b5ee-ddb2f74df14f)
  ✓ 양주시립회암사지박물관 (ID: 4c6ff5e2-4074-41dc-8c3d-75c789dcb55b)
  ✓ 달성공원 (ID: 532fdb98-a374-4283-a442-2a4240ea7696)
  ✓ 제주절물자연휴양림 (ID: 5333fd84-a0d5-4f45-bd00-29fda16f576e)
  ✓ 함덕해수욕장 (함덕 서우봉 해변) (ID: 53eddfdf-086c-410e-893c-20258343ac79)
  ✓ 대포주상절리 (ID: 561bfb31-e733-4901-8fbf-303b2fc844a1)
  ✓ 강서농산물도매시장 (ID: 56d81bd3-606a-473b-801c-faf2b0c2b55b)
  ✓ 한반도지형(강원고생대 국가지질공원) (ID: 5702c0b3-12e3-4321-95b5-81d13fd9a01b)
  ✓ 스플라스 리솜 (ID: 57892e71-93ca-4e78-8d39-3c8018243117)
  ✓ 순천 낙안읍성 (ID: 5ddc90b5-645e-4d7f-851d-7d49adc660b1)
  ✓ 이호테우해변 (ID: 5ea8d513-cffc-4993-abad-197d736651da)
  ✓ 정방폭포 (ID: 5eda9ced-5f76-4a22-9afc-bed01952c370)
  ✓ 경주 문무대왕릉 (ID: 6092adec-92b2-4eb1-a875-fb5513d9a305)
  ✓ 백길해변 (ID: 609e5df5-3354-4b9b-b3e3-3ba067cb857e)
  ✓ 용두암 (ID: 619aecc2-fb2e-41a7-97bc-f475c79d7264)
  ✓ 보령 천북굴단지 (ID: 6403d9f9-9645-4b2a-856f-de498f72b5b4)
  ✓ 돌산공원 (ID: 67c8cefb-2648-494d-a1cd-490e7d845bee)
  ✓ 수원 화성 [유네스코 세계유산] (ID: 67e459d1-7cab-41e6-a651-32020ae1ed80)
  ✓ 구례 섬진강 벚꽃길 (ID: 6829f9d2-d24f-40f3-9be3-c97a177c0602)
  ✓ 월드컵공원 (ID: 6a59bb86-f4d1-4e63-97e7-671479ca32a5)
  ✓ 수목원길 야시장 (ID: 6bef2172-3f88-41be-b0f7-ee8b82235289)
  ✓ 광명동굴 (ID: 6e03a5b1-51f6-4679-a0e1-2a61f395c4c8)
  ✓ 올림픽체조경기장(KSPO DOME) (ID: 6f34d3a6-a8b4-4950-8893-e2591358d226)
  ✓ 경동시장 (ID: 6f9c69df-69e8-4dec-ba27-c31789310df3)
  ✓ 통영중앙전통시장 (ID: 70630e73-c3e2-4b7d-baa5-2e64bd972730)
  ✓ 유엔기념공원 (ID: 70a80bd0-67b1-4537-ba17-b18e091fc12a)
  ✓ 유앤아이센터 (ID: 71b282d0-d66c-4f67-bea6-a65f7bbd3324)
  ✓ 수원 남문시장 (ID: 71eabd87-5109-4b84-8198-6fa8e22c3f92)
  ✓ 모란민속5일장 (ID: 72e3f699-f78a-47b0-977a-b0b66cc0b107)
  ✓ 비슬산자연휴양림 (ID: 741cc1ef-8394-4ae0-8f72-ea4b1b249a63)
  ✓ 악양생태공원 (ID: 74469b45-d5d1-4697-9298-cbd306064155)
  ✓ 씨라이프부산아쿠아리움 (ID: 74efd03d-9097-4a9a-8ec0-3c813fafd2af)
  ✓ 도산서원 [유네스코 세계유산] (ID: 750ea8b1-7ac3-4067-ad58-592500295824)
  ✓ 와온해변 (ID: 75d0a3f2-157e-4a10-a1df-015bac9d67a2)
  ✓ 롯데월드타워&롯데월드몰 (ID: 762e796f-d214-4d79-b777-c6183cef1c73)
  ✓ 용마폭포공원 (ID: 77bd2359-4af0-4322-b2fa-f92e59151b36)
  ✓ 진천 농다리 (ID: 785e6b1b-ecef-4187-b081-5c417d7ee3b7)
  ✓ 문경단산 모노레일 (ID: 7b21ee0a-6803-4263-a90d-e1f69a10370d)
  ✓ 올림픽공원 (ID: 7ba5e7f2-5b78-43a1-8806-d8a6c0e7af2c)
  ✓ 외암민속마을 (ID: 7f196bdb-9967-4322-b1b0-ba6f9b3ec6c3)
  ✓ 서울고속버스터미널 꽃도매상가 (ID: 7f971427-8dc4-44f6-aa3e-2ace057b9066)
  ✓ 인천 차이나타운 (ID: 7f9d2672-2325-4d0f-8d3b-b104f47555cb)
  ✓ 병방치 스카이워크 (ID: 805e70f6-9390-4558-98a1-c59e0a0e2781)
  ✓ 파주놀이구름 (ID: 82bcee35-001e-4cfc-99cf-21a244aed260)
  ✓ 헤이리 예술마을 (ID: 82d9a397-b8c5-454e-bb71-dff610fb9bb9)
  ✓ 수덕사(예산) (ID: 845d1bda-d8f5-41a4-8ed8-d2ae745f3913)
  ✓ 남한산성도립공원 [유네스코 세계유산] (ID: 857387b2-ca18-483d-88aa-e21809c04b17)
  ✓ 더현대 서울 (ID: 866c034c-1648-4e0f-a218-be9709ec9057)
  ✓ 칠포해수욕장 (ID: 872f99a5-18f6-401c-bbe5-a54675feeb1c)
  ✓ 국제시장 (ID: 87453280-9034-4d13-ac02-52cc085cffef)
  ✓ 한밭수목원 (ID: 88e99ca3-8e16-469c-a753-45ecba45573e)
  ✓ 호텔 농심 (ID: 895217e3-fb5c-4ed7-88ce-5ad87b450f8a)
  ✓ 춘천 삼악산 호수케이블카 (ID: 8abcb54d-20cc-4712-99c0-2ccbc5370c51)
  ✓ 서해랑 제부도 해상케이블카 (ID: 8c6edec0-af2b-413c-8e3d-1363b3d89620)
  ✓ 낙강물길공원 (ID: 8ff4bcb5-e404-4329-a6bd-75077bcc8cec)
  ✓ 서귀포매일올레시장 (ID: 902a5c1e-f645-48fa-99e2-9fd7b310736c)
  ✓ 퍼스트가든 (ID: 9290abf1-d7ca-4c1d-a4d1-352ec5693726)
  ✓ 경주 석굴암 [유네스코 세계유산] (ID: 92bc2ee6-ac19-4a93-aa46-e54300fb4920)
  ✓ 산굼부리 (ID: 92bc4c6f-9f8e-4519-9150-f7b5fe6f32b5)
  ✓ 이가리 닻 전망대 (ID: 92c780f2-968e-4fd9-9ce7-7aaf167b70a4)
  ✓ 이충레포츠공원 (ID: 9612da7b-b21b-4bba-a587-233d76b8b095)
  ✓ 청주 상당산성 (ID: 967820c7-0969-41c4-b1ae-0d515f054458)
  ✓ 대구수목원 (ID: 96b2d98d-4c5e-41c4-b436-08a682f39165)
  ✓ 물의 정원 (ID: 9709dda6-1ce0-4548-88c9-810fe37a229d)
  ✓ 청풍리조트 (ID: 973a81a4-ba07-4246-824b-a2cd3969e493)
  ✓ 제부도 (ID: 97617fa7-09ca-434c-a2c3-df1bebd4c748)
  ✓ 시흥 갯골생태공원 (ID: 9aa7f780-1032-43b1-aa9d-12c26b5df4a2)
  ✓ 킨텍스 바이 케이트리 (ID: 9b49b2ac-e83a-4832-ba5b-fe6ef166620c)
  ✓ 롯데아울렛 남악점 (ID: 9d8fb5b4-f24f-4564-be1a-0054f9b3c285)
  ✓ 강문해변 (ID: 9eb45e26-cd7e-4d50-bffc-fecad800ce68)
  ✓ 프로방스 마을 (ID: 9ebc227a-584a-4df7-aec8-608bc5fea446)
  ✓ 용인농촌테마파크 (ID: 9f5f87a4-e3a3-4eb3-abbf-f1524a5bf101)
  ✓ 해운대 달맞이길 (ID: 9f8f3f84-597c-4c13-8a5c-b92ef8d2cbbc)
  ✓ 제주 무지개해안도로 (ID: a03f59d0-bb99-4f50-9662-bb3fd25509b0)
  ✓ 정선5일장 / 정선 아리랑시장 (2, 7일) (ID: a183f4f7-d8e7-4f8b-861a-99bfcb993693)
  ✓ 수종사 (ID: a233fd0f-5605-4929-b906-2dc6abc696f6)
  ✓ 경주 황리단길 (ID: a241a148-3947-4b3b-a901-ef79bcf7eee7)
  ✓ 물향기수목원 (ID: a63fc20e-3553-476d-8440-fc86715c4a02)
  ✓ 사계해변 (ID: a7c33d48-a351-4ffe-aa68-1d020c523b55)
  ✓ 인스파이어 엔터테인먼트 리조트 (ID: a82d3064-2679-439f-8c62-5bdaf6da03bc)
  ✓ 을왕리해수욕장 (ID: a9f70061-ccc7-492a-a2aa-1c5da4958ab1)
  ✓ 소노벨 비발디파크 (ID: ab54b483-f98c-4702-984a-e11a463814c3)
  ✓ 묵호항 (ID: ac2c6b14-ec5f-469a-ac28-29095c44bd09)
  ✓ 강화 동막해변 (ID: ac51fd3d-481b-4883-a11a-55f8f6efb3a9)
  ✓ 남산서울타워 (ID: ad931739-fca3-4135-a878-cc7fb82920ba)
  ✓ 장안사(부산) (ID: ae0702f3-1131-4cdd-930f-c7ce81831441)
  ✓ 산지천 (ID: ae2f62b2-72f2-480f-b951-3af744c8aa34)
  ✓ 제주민속촌 (ID: ae800d13-cc8e-4b5b-b809-f6e660ab755c)
  ✓ 해수워터피아 (ID: b1f27748-dd66-4c24-8d23-cdad86afb570)
  ✓ 하슬라아트월드 (ID: b365adfd-aa55-46c0-b1bb-a036f508a12d)
  ✓ 안양농수산물도매시장 (ID: b447803e-7819-4750-aed4-571df5ba232f)
  ✓ 동촌유원지 (ID: b4ffdbfd-82fd-49ad-90ba-58b8b3c0eb2e)
  ✓ 제주 산방산탄산온천 (ID: b55f95bc-e11c-430f-a24c-1d04f8ec23cc)
  ✓ 월미도 (ID: b899a06e-127b-402b-933b-65bc64a303e0)
  ✓ 청풍호 관광모노레일 (ID: b9d03d6a-83f6-4772-9a3f-52fe7ef36bbd)
  ✓ 김녕성세기해변 (김녕해수욕장) (ID: ba131bd3-10a4-4e4d-9440-442c728e48c3)
  ✓ 천호지·단대호수 (ID: ba5ea2af-687f-4406-947c-179a3113b983)
  ✓ 월정사·월정사 전나무숲 (ID: bc7f36d9-5b2f-4a5a-8464-0a830e4c4f71)
  ✓ 9.81 파크 제주 (ID: bfb089cb-5cd6-4728-9502-11a23743cd47)
  ✓ 화성 융릉(장조·헌경황후)과 건릉(정조·효의황후) [유네스코 세계유산] (ID: c115e06a-ae2f-4d2f-8f88-41f7d635b888)     
  ✓ 섭지코지 (ID: c2b1e89f-cc48-46f4-b15b-ccf24db7776c)
  ✓ 서울 암사동 유적 (ID: c3a6b30b-4b53-48f3-a6d0-c46dfc606d2d)
  ✓ 당진 삽교호 (ID: c58680b5-e48b-4e92-9255-43794b7aa93e)
  ✓ 사천바다 케이블카 (ID: c8664819-5f7d-429f-a390-124c1d5cc390)
  ✓ 국립전주박물관 (ID: c8f4652a-bba8-423e-885d-ae8cc704b588)
  ✓ 조계사(서울) (ID: cb481702-e012-46c6-a806-cbc9c3c8c8dd)
  ✓ 와우정사(용인) (ID: cf1a5c90-99dc-416e-8570-08c76ac5d712)
  ✓ 속초관광수산시장 (ID: cf3c29c0-0fbe-44e2-8f2f-c8ff92f781a0)
  ✓ 포항 죽도시장 (ID: d130e154-1c06-4949-a811-16929fef1af5)
  ✓ 주문진항 (ID: d2b10d5d-f2eb-4658-ab4e-2fe431a82836)
  ✓ 국립현대미술관 서울 (ID: d51b6e84-e905-4901-955f-ab4182f108a5)
  ✓ 영종도 (ID: d5e69659-453f-457a-9211-654e66d80f6a)
  ✓ 추억의 달동네 (ID: d6b99d2c-0c67-4ca9-86f6-04288c521c91)
  ✓ 화담숲 (ID: d910e258-111d-43f1-8d29-80f4e80ca92a)
  ✓ 대구 칠성시장&별별상상 칠성야시장 (ID: da15d47f-c0b6-4cb0-9f91-3e9e5c217ce2)
  ✓ 궐리사 (ID: dcd09c68-951a-4818-93ba-8b7331c80665)
  ✓ 호암미술관 (ID: ddceb6ae-7a32-4838-b5d9-7f4d07c8a4c3)
  ✓ 전주동물원 (ID: dd4f57bb-38df-4c78-9fec-add907177dbe)
  ✓ 강화루지(강화씨사이드리조트) (ID: df3d3000-6c71-431d-a05a-2b426a49a20e)
  ✓ 협재해수욕장 (ID: e00d3013-55ef-4c44-b944-571c50bc4b88)
  ✓ 순천 아랫장 (2, 7일) (ID: e0d89111-b8ab-4a1f-a35e-d1c480ee271e)
  ✓ 장생포 고래박물관 (ID: e1635bb9-938b-4a91-bb7b-e0cbb25d30e3)
  ✓ 전류리포구 (ID: e2c6e613-86cf-4d75-8ac8-9d3aa999bdd9)
  ✓ 마니산(강화) (ID: e2f4b541-06eb-4607-8c67-5187a0320375)
  ✓ 아쿠아플라넷 제주 (ID: e32c4817-5b4e-4f70-b5da-1d3561a0bc78)
  ✓ 카멜리아힐 (ID: e4f4a116-be4b-46fa-a289-11e223fa597a)
  ✓ 하이원 그랜드호텔 (ID: e68cdd27-7532-4b92-92de-a8dc51da4f41)
  ✓ 율암온천숯가마 테마파크 (ID: e9cef255-6c9f-43f0-8c90-6dadb484a24b)
  ✓ 국립생태원(서천) (ID: ea240cef-8865-4419-a077-931a60e9ca89)
  ✓ 쁘띠프랑스 (ID: eadf5f3e-81f9-4da8-9346-5eba291ae970)
  ✓ 공주 공산성 [유네스코 세계유산] (ID: eb573856-bcbb-4934-a063-06dfaaf8a961)
  ✓ 오색약수터 (ID: ec03c7e3-8400-44eb-b340-40ede615447e)
  ✓ 주산지(청송 국가지질공원) (ID: efe9590a-a0ef-48d2-98c3-8959f5cded5b)
  ✓ 중문색달해수욕장 (ID: f0488695-d42b-4b5f-9772-73ad6865a5e8)
  ✓ 서울 명동성당 (ID: f12eeeee-67cd-4861-a067-fcafc7d3e6e9)
  ✓ 선유도 (ID: f172269f-915a-4b56-a137-eacf3e92338c)
  ✓ 대부도 (ID: f2c2a0a0-7e07-4682-932b-017a3d803d09)
  ✓ 김포 대명항 (ID: f2f2cfec-8c21-4044-88e9-59ec42a8e3a5)
  ✓ 향일암(여수) (ID: f31fbd7d-2a91-46ad-9c5e-4a5511422a45)
  ✓ 설해원 (ID: f5196f7f-4f80-461e-8888-f3effd1bee93)
  ✓ 낙산해수욕장 (ID: f58b6922-b961-4464-a746-aba2e44b3ac0)
  ✓ 동문재래시장 (ID: f598268e-368c-4e52-8f5a-7e8d3dc8b32a)
  ✓ 대천해수욕장 (ID: f5e68f00-14b4-449b-94ae-5a2640cbdb08)
  ✓ 군산근대역사박물관 (ID: f5ea8592-e85b-43e9-8395-29190eb78906)
  ✓ 포천아트밸리 (한탄강 유네스코 세계지질공원) (ID: f6dac219-e397-4e01-870d-a508aed4397e)
  ✓ 안목해변 (ID: f9d3fd78-55bd-4d5c-90fe-25af7ffa76a0)
  ✓ 대포항 (ID: fab79ef5-d21e-4c3b-bf31-9fc5997ed400)
  ✓ 중앙탑공원(충주) (ID: fce1103b-1340-4bb9-9230-f405ce71d754)
  ✓ 제천 중앙시장 (ID: ffa75747-17bb-4526-a0c9-e0b627234cff)
  ✓ 순담계곡 (ID: 027d37f9-5499-4925-92b6-663c7c58a085)
  ✓ 율동공원 (ID: 04425db6-a32c-444d-8eda-336a58c6aa33)
  ✓ 원당시장 (ID: 0457cdd7-6fd2-4510-8135-692e984cc960)
  ✓ 장생포 고래문화마을 (ID: 0460fc0b-711d-491f-92c3-a70481371ae3)
  ✓ 스타필드 고양 (ID: 048220a0-be4c-4ee0-aec4-eb1e323b3c50)
  ✓ 김해수로왕릉 (ID: 04f41d21-459a-4112-8947-3c77d3eedfdc)
  ✓ 롯데몰 진주점 (ID: 0583e8f3-a975-4f82-b7b3-a913d6e67815)
  ✓ 코오롱호텔 (ID: 05aac895-1b95-460c-8f54-7914e87cc606)
  ✓ 서피비치 (ID: 05f647eb-db7f-4308-9b3c-2989254f8aba)
  ✓ 마이산 벚꽃길 (ID: 06822469-35dd-4423-8601-51ead5a33998)
  ✓ 서머셋 제주신화월드 (ID: 0790057f-23dd-4c4b-85f6-0c0f5492b034)
  ✓ 동대구역광장 (ID: 0e4d71a4-7608-45eb-820c-b6698f13505e)
  ✓ 부산광역시남구시설관리공단 남구빙상장 (ID: 0f661189-ae96-4ce6-b221-f34c1338fad6)
  ✓ 킨텍스 (ID: 11661754-7237-40c8-a771-ea6ef50e30db)
  ✓ 인천국제공항 제2여객터미널 (ID: 1178af39-c92b-4f27-9599-0abae9024989)
  ✓ 홍련암 (ID: 1178b349-a081-4e7f-8b57-126cdda26ac1)
  ✓ 롯데월드 아쿠아리움 (ID: 11fdf5c3-94d2-4b6b-aee9-6896a331ce76)
  ✓ 삼천포 중앙시장 (ID: 13103f75-362b-4fc9-8ae9-adf41062aa99)
  ✓ 고하도 전망대 (ID: 14375fc6-24da-43af-a98d-1b7adac76491)
  ✓ 광명시민체육관 (ID: 15549de6-30fb-4b81-92de-d9c5d0978d88)
  ✓ 서울 신림동 민속순대타운 (ID: 15917c7e-0c2d-45ae-8d78-ea9d90ecb1a9)
  ✓ 키자니아 서울 (ID: 1664bf82-9c75-48f4-b926-b445a1aa9f79)
  ✓ 이천설봉온천랜드 (ID: 19ca13bf-a861-48a4-9bd7-528f5ecb672b)
  ✓ 청량사(봉화) (ID: 19d5c259-8315-4430-8228-4755db5ec27b)
  ✓ 아그로랜드 태신목장 (ID: 19e1d49a-3464-4529-a0f0-7e50f8a1579b)
  ✓ 점핑파크 인천계양점 (ID: 1a1593f5-c119-484b-9ab2-a1d6890e9a67)
  ✓ BIFF 광장 (ID: 1a38892a-6671-4236-adc3-7036ce10c2b2)
  ✓ 남원시립김병종미술관 (ID: 1ce00ea6-9174-4bd5-97f7-19309e20a124)
  ✓ 무등산국립공원(광주) (ID: 1e4a7c8f-55d0-4d20-8381-01747ebf14c9)
  ✓ 춘천 애니메이션박물관&토이로봇관 (ID: 1f91dc19-b946-4f02-8cf3-af09e27a54e0)
  ✓ 상소동 산림욕장 (ID: 2117bd9b-c0a4-411e-802d-a71e8e8b122e)
  ✓ 국립횡성숲체원 (ID: 2130956f-9615-4af6-9cf9-f5779962bff9)
  ✓ 오산 미니어처빌리지 (ID: 227512f6-9ed9-4d51-b93b-d87921f6c440)
  ✓ 메종 글래드 제주 (ID: 22c2ecdb-2d29-45bb-96c4-59f2e853e54c)
  ✓ 더현대 대구 (ID: 22cb53f9-0223-43a3-a0f7-ed0faaf8e6bb)
  ✓ 백운계곡 (한탄강 유네스코 세계지질공원) (ID: 23915870-4b1a-4dfa-9c6a-65e06afda089)
  ✓ 고양어린이박물관 (ID: 24993f58-d65a-4720-8250-c86da836c194)
  ✓ 해운대 블루라인파크 (ID: 2670917c-c7b0-43be-a174-bd5ae40372f1)
  ✓ 더 케이 호텔 경주 (ID: 26c7c85f-5d53-408b-bf2b-34e886d067ba)
  ✓ 과천향교 (ID: 27866997-cf31-47b9-97d7-ed4fafea5d6b)
  ✓ 촉석루 (ID: 295cfed1-6be9-44b1-af63-a156e8cc06a9)
  ✓ 이끼터널 (ID: 29677511-10ce-471d-8fdd-b26d04873f6b)
  ✓ 반포한강공원 (ID: 29dc393d-367f-4721-b283-0067bf76ffd4)
  ✓ 궁평항 (ID: 2a099a85-cc53-41bc-9d34-44830fd6f643)
  ✓ 피나클랜드 수목원 (ID: 2bbcac3a-cf71-4f3a-ad42-a1061f6d06be)
  ✓ 변산반도국립공원 (ID: 2d937ac8-5482-4aa5-8b30-b3c5e70ee13f)
  ✓ 숨도 (ID: 2f49a3f2-e03f-493e-b690-a6b6548f15ea)
  ✓ 국립백두대간수목원 (ID: 30f0a401-5c26-46c5-9fb7-7d3de7da4056)
  ✓ 무섬마을 (ID: 326111bb-6df9-4242-a5c7-527a3381d46f)
  ✓ 롯데시티호텔 대전 (ID: 336e56bb-6d0e-43b5-8b6f-62f7cd325722)
  ✓ 가곡유황온천&스파 (ID: 352219ef-90e0-44c0-af63-f99a691a77aa)
  ✓ 센트럴시티 (ID: 3558b30c-d1c8-4031-bb57-de494d364b93)
  ✓ 제주불빛정원 (ID: 3730c867-bef7-44ca-b251-5097b2d306dc)
  ✓ 청계천 (ID: 382d5823-16f0-4793-aea7-3e77451bd68e)
  ✓ 송도스카이파크 (ID: 3942525b-79fe-4e71-a76c-d292955ba313)